# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#export csv to DataFrame

weather2_df = pd.read_csv("my_weather_data.csv")
weather2_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.65,115.33,52.00,88,56,10.04,AU,1594930776
1,1,Deputatsky,69.30,139.90,40.24,79,34,1.36,RU,1594930935
2,2,São João da Barra,-21.64,-41.05,71.60,68,75,6.93,BR,1594930935
3,3,Vaini,-21.20,-175.20,77.00,78,20,6.93,TO,1594930935
4,4,Tasiilaq,65.61,-37.64,48.20,66,100,13.87,GL,1594930935


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#store latitude and longitude in a variable
locations = weather2_df[["Latitude", "Longitude"]]

#convert humidity to float
humidity = weather2_df["Humidity"].astype(float)
weather2_df.count()


Unnamed: 0    545
City          545
Latitude      545
Longitude     545
Max Temp      545
Humidity      545
Cloudiness    545
Wind Speed    545
Country       542
Date          545
dtype: int64

In [4]:
#set size of map to be displayed
figure_layout = {
    'width': '420x',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px'
}

#configure settings of map for display and layout
gmaps.figure(map_type='TERRAIN', center=(48.0,0), zoom_level=12, layout=figure_layout)
fig = gmaps.figure()

#set and add heat layer (humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
             dissipating=False, 
             max_intensity = 50, point_radius = 1.0)
fig.add_layer(heat_layer)

#display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#define ideal weather conditions
ideal_temp = (weather2_df["Max Temp"]>=70) & (weather2_df["Max Temp"]<=80)
ideal_wind = weather2_df["Wind Speed"] <15
ideal_clouds= weather2_df["Cloudiness"] <10
ideal_vacation = ideal_temp&ideal_wind&ideal_clouds

#create new dataframe
vacation_df = weather2_df[ideal_vacation]
#drop null or blank rows
vacation_df.dropna()

vacation_df.head(10)


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,Corinth,37.94,22.96,77.00,58,0,3.04,GR,1594930343
33,33,Avarua,-21.21,-159.78,71.60,73,0,9.17,CK,1594930943
54,54,Ribeira Brava,32.65,-17.07,73.40,69,1,4.70,PT,1594930948
71,71,Ponta do Sol,32.67,-17.10,73.40,69,1,4.70,PT,1594930951
101,101,Province of Mantua,45.17,10.78,75.99,68,1,4.68,IT,1594930957
186,186,Luanda,-8.84,13.23,71.60,88,0,9.17,AO,1594930911
216,216,Vallenar,-28.57,-70.76,77.07,18,0,4.94,CL,1594930872
223,223,Misratah,32.38,15.09,79.21,77,0,13.53,LY,1594930973
245,245,Tūkrah,32.53,20.58,76.87,67,0,5.41,LY,1594930979
247,247,Soc Trang,9.60,105.98,77.95,87,2,2.35,VN,1594930979


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#store into varioable hotel_df
hotel_df = vacation_df
#add a "Hotel Name" column to the dataframe
hotel_df['Hotel Name'] = ""

         

C:\Users\aolan\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
#start building url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set parameters for search
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [8]:
#use iterrows to loop through the dataframe

for index, row in vacation_df.iterrows():
    city_name = row['City']
    Country = row['Country']
    lat = row["Latitude"]
    long = row["Longitude"]
# add keyword to params dict
    params["location"] = f"{lat},{long}"
    
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name = response['results'][0]['name']
    
        print(f"Closest hotel to {city_name} is {hotel_name}.")
              
        print("------------")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except (KeyError, IndexError):
        print("Unable to find result")
        
        print("------------")
    
    
    

Closest hotel to Corinth is Hotel King Saron Ξενοδοχείο.
------------


C:\Users\aolan\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel to Avarua is The Rarotongan Beach Resort & Lagoonarium.
------------
Closest hotel to Ribeira Brava is Estalagem Da Ponta Do Sol.
------------
Closest hotel to Ponta do Sol is Estalagem Da Ponta Do Sol.
------------
Closest hotel to Province of Mantua is Casa Museo Palazzo Valenti Gonzaga.
------------
Closest hotel to Luanda is EPIC SANA Luanda.
------------
Closest hotel to Vallenar is Hotel Origenes.
------------
Closest hotel to Misratah is Safari Hotel.
------------
Closest hotel to Tūkrah is قاعدة فيقا فيقا.
------------
Closest hotel to Soc Trang is Khách Sạn Xuân Huỳnh 3.
------------
Closest hotel to Bertinoro is Hotel Fontanelle.
------------
Closest hotel to Aiquile is HOSTAL "PLAZA".
------------
Closest hotel to Forio is Albergo Della Regina Isabella.
------------
Closest hotel to Henties Bay is Desert Rose.
------------
Closest hotel to Kiryat Gat is Desert Gat.
------------
Closest hotel to Salta is Villa Vicuña Hotel Boutique.
------------
Closest hotel to

In [9]:
hotel_df.head()
    


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,20,Corinth,37.94,22.96,77.00,58,0,3.04,GR,1594930343,Hotel King Saron Ξενοδοχείο
33,33,Avarua,-21.21,-159.78,71.60,73,0,9.17,CK,1594930943,The Rarotongan Beach Resort & Lagoonarium
54,54,Ribeira Brava,32.65,-17.07,73.40,69,1,4.70,PT,1594930948,Estalagem Da Ponta Do Sol
71,71,Ponta do Sol,32.67,-17.10,73.40,69,1,4.70,PT,1594930951,Estalagem Da Ponta Do Sol
101,101,Province of Mantua,45.17,10.78,75.99,68,1,4.68,IT,1594930957,Casa Museo Palazzo Valenti Gonzaga


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig




Figure(layout=FigureLayout(height='420px'))